In [29]:
!pip install -q pdfquery
!pip install -q PyPDF2
!pip install -q openai
!pip install -q python-dotenv

In [30]:
import pandas as pd
import pdfquery
from pdfquery import PDFQuery
import PyPDF2
import re
import os
import openai
from dotenv import load_dotenv

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
folder_path = '/content/drive/MyDrive/LLM_RESUME_DATA/All Resumes/'

In [35]:
load_dotenv('/content/config.env')

True

In [36]:

def extract_text_from_pdf(file_path):
  with open(file_path, 'rb') as file:
      reader = PyPDF2.PdfReader(file)
      text = ''
      for page in reader.pages:
          text += page.extract_text()
      return text
def clean_text(text):
  # Remove extra spaces, newlines, and special characters
  cleaned_text = re.sub(r'\s+', ' ', text)
  return cleaned_text.lower()

def extract_sections(text):
  client = openai.OpenAI(
    api_key=os.getenv('TOGETHER_API'),
    base_url="https://api.together.xyz/v1",
  )

  response = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    messages=[
      {"role": "system", "content": '''You are a system that structure the resume in following sections. Introduction, Work_Experience, Skills, Project, Achievements, Hobbies and Additional_Info. Use following tags for structing the information:
      1. <Introduction> </Introduction>
      2. <Work_Experience> <Place> </Place> <Work_Experience>
      3. <Skills> <list> </list> </Skills>
      4. <Project> <title> </title> </Description> </Project>
      5. <Achievement> <list> </list> </Achievement>
      6. <Hobbies> <list> </list> </Hobbies>
      7. <Additonal_info> </Additional_info>
      '''},
      {"role": "user", "content": f"{text}"},
    ]
  )

  answer = str(response.choices[0].message.content)
  return answer


In [41]:
def create_structured_resume(folder_path):
  # listing all the folders in the 'All Resume' folder
  folders_in_all_resume = [f for f in os.listdir(folder_path)]
  total_resume_count = 0
  for folder in folders_in_all_resume:
    nested_folder_path = folder_path + f'{folder}/'
    for nested_folder in os.listdir(nested_folder_path):
      if (nested_folder == "resumes"):
        nested_resume_folder_path = nested_folder_path + f'resumes/'
        for resume in os.listdir(nested_resume_folder_path):
          # check if the resume is in the pdf format
          if (resume[-4:]=='.pdf'):
            # if (total_resume_count > 944):
            try:
              extract_text = extract_text_from_pdf(nested_resume_folder_path+resume)
              cleaned_text = clean_text(extract_text)
              extracted_sections = extract_sections(cleaned_text)

              to_save_file = '/content/drive/MyDrive/LLM_RESUME_DATA/' + f"structured_resume/{total_resume_count}.txt"

              # Open the file in write mode and write the content to the file
              with open(to_save_file, 'w') as file:
                file.write(extracted_sections)
            except Exception as e:
              pass

            total_resume_count += 1


  print(f'Total number of resumes: {total_resume_count}')
  print("Done :)")

In [42]:
create_structured_resume(folder_path)

Total number of resumes: 2048
Done :)
